### Notebook for highlighting a bug in the `calibrate_energy_axis` method of the `EDSmodel` class in HyperSpy

The problem with the function is that the energy resolution is calibrated with only the first line given as the argument `xray_lines`, and as the default value of `xray_lines='all_alpha'`, the line used for reference is the alpabetically first one. This leads to a wrong/bad calibration of the energy resolution.

The energy resolution is calculated with the HyperSpy function `_get_sigma()`, which is from Fiori and Newbury (1978, see the docstring of `_get_sigma()`). This equation used the energy and FWHM of a reference line to estimate any other FWHM in the spectrum. If the reference line is a poorly defined one, the calibration will be wrong.

This is the results generated 9.2.2023 with HyperSpy 1.7.4, where it is clear that using Ga_La and As_La provides the most stable calibration.

```python
Used reference            30 kV      15 kV      10 kV       5 kV
original_calibration      130.0      130.0      130.0      130.0
all_alpha                 139.4      149.7      132.5      132.8
all_alpha_calibrated      138.3      148.8      132.0      132.2
As_Ka                     137.9      146.4        nan        nan
As_La                     130.0      130.4      131.9      132.1
Ga_Ka                     132.9      131.7      668.3        nan
Ga_La                     129.7      130.9      127.4      130.8
```

When using `'all_alpha'`, the 30 kV and 15 kV spectrum use As_Ka as reference, where As_Ka is not as well defined as the three other alpha lines. This is especially clear for the 15 kV spectrum.

The effect of a poorly defined line is emphasized by the Ga_Ka line calibration for 10 kV, which is >660 eV since the line is almost not defined at all.

The 10 and 5 kV spectrum have more stable calibration, as they both use As_La as reference when using `'all_alpha'`.

#### Improvements to HyperSpy

To fix this bug, an issue was opened on the HyperSpy GitHub page [here](https://github.com/hyperspy/hyperspy/issues/3098).

As of now (9.2.2023) a documentation change have been suggested in [this PR](https://github.com/hyperspy/hyperspy/pull/3099), and a code change might be suggested in the future.

In [ ]:
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
path = 'data/Maehlum_2022-09-06_EDS-SEM-APREO'
file30 = 'GaAs_30kV.emsa'
file15 = 'GaAs_15kV.emsa'
file10 = 'GaAs_10kV.emsa'
file05 = 'GaAs_05kV.emsa'
files = [file30, file15, file10, file05]
# elements = ['Ga', 'As', 'O', 'C', 'Si']
elements = ['Ga', 'As']
zero_peak_end_index = 33

In [ ]:
# s30 = hs.load(path + '/' + file30, signal_type='EDS_SEM')
# s15 = hs.load(path + '/' + file15, signal_type='EDS_SEM')
# s10 = hs.load(path + '/' + file10, signal_type='EDS_SEM')
# s05 = hs.load(path + '/' + file05, signal_type='EDS_SEM')
# ss = [s30, s15, s10, s05]

ss = [hs.load(path + '/' + file, signal_type='EDS_SEM') for file in files]

for i in range(len(ss)):
    ss[i].add_elements(elements)
    # slice after zero peak and at nominal beam energy
    ss[i] = ss[i].isig[zero_peak_end_index:ss[i].metadata.Acquisition_instrument.SEM.beam_energy]


In [ ]:
models = []
for s in ss:
    m = s.create_model()
    m.fit()
    m.fit_background()
    models.append(m)

results = {}

In [ ]:
def vacc(s):
    """Get the accelerating voltage from the metadata"""
    return s.metadata.Acquisition_instrument.SEM.beam_energy

def mn_ka(s):
    """Get the Mn Ka energy from the metadata"""
    return np.round(s.metadata.Acquisition_instrument.SEM.Detector.EDS.energy_resolution_MnKa, decimals=1)

In [ ]:
def print_resolutions(list):
    for s in ss:
        # print(f'{vacc(s):4.0f} kV:  {mn_ka(s):.0f} eV')
        list.append(mn_ka(s))


In [ ]:
# print('Original resolutions: ')
original_resolutions = []
print_resolutions(original_resolutions)
results['original_calibration'] = original_resolutions

In [ ]:
# calibrate with xray_lines='all_alpha', which is the default value
def calibrate_all_alpha():
    for m in models:
        m.calibrate_energy_axis(calibrate='resolution', xray_lines='all_alpha')


In [ ]:
calibrate_all_alpha()
all_alpha_resolutions = []
print_resolutions(all_alpha_resolutions)
results['all_alpha'] = all_alpha_resolutions

In [ ]:
# calibrate all the xray lines, then doing the energy calibration
for m in models:
    m.calibrate_xray_lines(calibrate='energy', xray_lines='all', kind='multi', iterpath='serpentine')
    m.calibrate_xray_lines(calibrate='width', xray_lines='all', kind='multi', iterpath='serpentine')

calibrate_all_alpha()
all_alpha_resolutions_calibrated = []
print_resolutions(all_alpha_resolutions_calibrated)
results['all_alpha_calibrated'] = all_alpha_resolutions_calibrated
results

In [ ]:
# calibrating on As_Ka

models[0].calibrate_energy_axis(calibrate='resolution', xray_lines=['As_Ka'])
models[1].calibrate_energy_axis(calibrate='resolution', xray_lines=['As_Ka'])
# models[2].calibrate_energy_axis(calibrate='resolution', xray_lines=['As_Ka'])  # No As_Ka in 10 kV
# models[3].calibrate_energy_axis(calibrate='resolution', xray_lines=['As_Ka'])  # No As_Ka in 5 kV


results['As_Ka'] = [mn_ka(ss[0]), mn_ka(ss[1]), np.nan, np.nan]

In [ ]:
# calibrating on As_La
models[0].calibrate_energy_axis(calibrate='resolution', xray_lines=['As_La'])
models[1].calibrate_energy_axis(calibrate='resolution', xray_lines=['As_La'])
models[2].calibrate_energy_axis(calibrate='resolution', xray_lines=['As_La'])
models[3].calibrate_energy_axis(calibrate='resolution', xray_lines=['As_La'])

results['As_La'] = [mn_ka(ss[0]), mn_ka(ss[1]), mn_ka(ss[2]), mn_ka(ss[3])]

In [ ]:
# calibrating on Ga_Ka

models[0].calibrate_energy_axis(calibrate='resolution', xray_lines=['Ga_Ka'])
models[1].calibrate_energy_axis(calibrate='resolution', xray_lines=['Ga_Ka'])
models[2].calibrate_energy_axis(calibrate='resolution', xray_lines=['Ga_Ka'])
# models[3].calibrate_energy_axis(calibrate='resolution', xray_lines=['Ga_Ka'])  # No Ga_Ka in 5 kV


results['Ga_Ka'] = [mn_ka(ss[0]), mn_ka(ss[1]), mn_ka(ss[2]), np.nan]

In [ ]:
# calibrating on Ga_La
models[0].calibrate_energy_axis(calibrate='resolution', xray_lines=['Ga_La'])
models[1].calibrate_energy_axis(calibrate='resolution', xray_lines=['Ga_La'])
models[2].calibrate_energy_axis(calibrate='resolution', xray_lines=['Ga_La'])
models[3].calibrate_energy_axis(calibrate='resolution', xray_lines=['Ga_La'])

results['Ga_La'] = [mn_ka(ss[0]), mn_ka(ss[1]), mn_ka(ss[2]), mn_ka(ss[3])]

In [ ]:
# the alphabetically first line is used as reference, which is:
print('all_alpha calibration uses the following line as reference')
for m in models:
    print(str(m)[-5:-1], str(m.xray_lines[0])[1:6])

In [ ]:
print(f'{"Used reference":20} {"30 kV":>10} {"15 kV":>10} {"10 kV":>10} {"5 kV":>10}')
for k, v in results.items():
    print(f'{k:20} {v[0]:10.1f} {v[1]:10.1f} {v[2]:10.1f} {v[3]:10.1f}')

In [ ]:
[l.name for l in m.xray_lines]

In [ ]:
# make a table with counts in the lines

headers = ['Vacc', 'As_Ka', 'As_La', 'Ga_Ka', 'Ga_La']
Vaccs = ['30 kV', '15 kV', '10 kV', ' 5 kV']
rows = [headers]
for i in range(len(models)):
    row = [Vaccs[i]]
    for line in headers[1:]:
        if line in [l.name for l in models[i].xray_lines]:
            row.append(f'{(models[i][line].A.value/1e5):.1f}')
            # row.append(f'{(models[i][line].A.value):.1e}')
            # row.append(f'{(models[i][line].A.value):.0f}')
        else:
            row.append('---')
    rows.append(row)

In [ ]:
print('Counts in the peaks, divided by 1e5')
for r in np.array(rows).T:
    row = ''
    for c in r:
        row += f'{c:>16}'
    print(row)

In [ ]:
delim = ' & '

In [ ]:
print('Energy resolution in eV')
print(f'{"Reference line":>15} {delim} {"30 kV":>15} {delim} {"15 kV":>15} {delim} {"10 kV":>15} {delim} {"5 kV":>15} \\\\')
for k, v in results.items():
    # if k in ['original_calibration', 'all_alpha', 'all_alpha_calibrated']:
    if k in ['all_alpha']:

        continue
    print(f'{k[:15]:>15} {delim} {v[0]:>15.1f} {delim} {v[1]:>15.1f} {delim} {v[2]:>15.1f} {delim} {v[3]:>15.1f} \\\\')

In [ ]:
# replace "_" in a string with whitespace
xraylines[0][0].replace('_', ' ')

In [ ]:
xraylines = []
familylines = []
for i in range(len(models)):
    xraylines.append([l.name.replace('_', ' ') for l in models[i].xray_lines])
    familylines.append([l.name.replace('_', ' ') for l in models[i].family_lines])
xraylines
familylines

In [ ]:
# check if all lines are present in all models, with models[0] as reference
# if not, add a "----" as value to xraylines and familylines at the corresponding position
for i in range(len(models)):
    for j in range(len(xraylines[0])):
        if xraylines[0][j] not in xraylines[i]:
            xraylines[i].insert(j, '-----')
    for j in range(len(familylines[0])):
        if familylines[0][j] not in familylines[i]:
            familylines[i].insert(j, '-----')



In [ ]:
print('Lines in the models')
columns = 10
print(f'{"Model ":>11} {"&       "*columns}\\\\')
print(f'{"X-ray lines ":>11} {"&       "*columns}\\\\')
for i in range(len(models)):
    print(f'{vacc(ss[i]):10.0f} {delim} { " & ".join(xraylines[i])} {"&       "*6} \\\\')
print(f'{"Family lines ":>11} {"&       "*columns}\\\\')
for i in range(len(models)):
    print(f'{vacc(ss[i]):10.0f} {delim} { " & ".join(familylines[i])} \\\\')


In [ ]:
models[0].calibrate_energy_axis(calibrate='resolution', xray_lines=['As_Ka', 'As_La', 'Ga_Ka', 'Ga_La'])
mn_ka(ss[0])

In [ ]:
models[0].calibrate_energy_axis(calibrate='resolution', xray_lines=['As_Ka', 'As_La', 'Ga_Ka', 'Ga_Kb', 'Ga_La'])
mn_ka(ss[0])

In [ ]:
models[0].calibrate_energy_axis(calibrate='resolution', xray_lines=['Ga_La', 'As_Ka', 'As_La', 'Ga_Ka', 'Ga_Kb'])
mn_ka(ss[0])

In [ ]:
xraylines = [l.name for l in models[0].xray_lines]
familylines = [l.name for l in models[0].family_lines]


In [ ]:
lines = xraylines + familylines
lines

In [ ]:
models[0].calibrate_energy_axis(calibrate='resolution', xray_lines=lines)
mn_ka(ss[0])

In [ ]:
# Swap index 0 with index 3 in lines
lines[0], lines[3] = lines[3], lines[0]


In [ ]:
lines

In [ ]:
models[0].calibrate_energy_axis(calibrate='resolution', xray_lines=lines)
mn_ka(ss[0])

In [ ]:
models[0].calibrate_energy_axis(calibrate='resolution', xray_lines='all_alpha')
mn_ka(ss[0])

In [ ]:
models[0].calibrate_energy_axis(calibrate='resolution', xray_lines=['Ga_La'])
mn_ka(ss[0])

In [ ]:
models[0].calibrate_energy_axis(calibrate='resolution', xray_lines=['As_Ka', 'Ga_Kb'])
mn_ka(ss[0])